## Experimental Settings

In [ ]:
username = "sgt43" # Username of user on server
exp_name = "dsef-janus" # Name of Experiment on TestBed
proj_name = "cops" # Name of Project on TestBed
default_node = "node-0" # Default node to run the experiment from
ds = "janus"
host_ip = "users.emulab.net"
host_servers_ip = "emulab.net"
default_ip = "{}.{}.{}.{}".format(default_node, exp_name, proj_name, host_servers_ip)

## Imports / Definitions

In [ ]:
import yaml
import os
from importlib import reload
from altair import *
from altair_widgets import interact_with
import pandas as pd
import pydsef
from pydsef import Experiment, DSEFLoader, util

%load_ext yamlmagic
#TODO: do this in Jupyter config

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [ ]:
%%yaml conf --loader DSEFLoader
client_types:
    closed: &closed client_closed
    open: &open client_open
    open_5k: &open_5k client_open_5k
protocol:
    tapir: &tapir tapir:tapir
    janus: &janus brq:brq
    occ_paxos: &occ_paxos occ:multi_paxos
benchmark:
    rw: &rw rw_benchmark
    tpca: &tpca tpca
    tpcc: &tpcc tpcc
    tpcc_no: &tpcc_no tpcc_no
    tpcc_pm: &tpcc_pm tpcc_pm
client_placement:
    with_leader: &with_leader with_leader
    balanced: &balanced balanced
    
experiment:
    duration: 10
    name: dsef
    clients: !include [1, 2]
    client_loads: !include [-1]
    server_timeout: 30
    servers: !include [2]
    zipf: [0.0, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    shards: 3
    benchs: !include [*tpcc]
    protocols: !include [*tapir]
    client_placement: *balanced
    num_cores: 2              # number of cores each server has
    data_centers: []
    client_type: *closed # is client open or closed
    concurrent: 1             # concurrent: (1, 10, 25, 50, 100, 1000, 10000)
    overlap: True
    rpc_port: 5555
    status_time_interval: 5
    wait: False
    single_server: 0
    taskset_schema: 0
    client_taskset: False
    recording_path: ''
    interest_txn: 'NEW ORDER'
    
host:
    node-1: node-1
    node-2: node-2
    node-3: node-3
    node-4: node-4
    node-5: node-5
    node-6: node-6
    node-7: node-7
    node-8: node-8
    node-9: node-9
    node-10: node-10
    node-11: node-11
    node-12: node-12
    node-13: node-13
    node-14: node-14
    node-15: node-15
    node-16: node-16
    node-17: node-17
    node-18: node-18
    node-19: node-19
    node-20: node-20

In [ ]:
exp = Experiment(default_ip, username, ds, conf)
pd.DataFrame(exp.experiment_list)

# Experiment
Launch Instances of the Distributed System
    
Setup Benchmark on Clients
    
Start the Experiment
    
Kill all Instances of the Distributed System 

## Loop

In [ ]:
exp.set_executable('wrapper.py')
exp.push_files(['run.py', 'gen_experiments.py'])
exp.set_archive('dsef', 'log')

# exp.show_log = False
# exp.save_log = True
res = exp.run()

In [ ]:
data = {}
for exp_id in res.keys():
    tmp_data = {}
    for txn in res[exp_id].keys():
        txn_res = res[exp_id][txn].copy()
        
        for key in list(txn_res.keys()):
            if isinstance(txn_res[key], dict):
                for k in txn_res[key].keys():
                    txn_res['{}_{}'.format(key, k)] = txn_res[key][k]
                del txn_res[key]
                
        data['{}_{}'.format(exp_id, txn)] = txn_res

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
interact_with(df.transpose())